In [73]:
using JuMP, Gurobi
using BenchmarkTools
using Base.Test

In [98]:
module cj2

using Interfaces
using JuMP
using Unrolled

import Base: show, hash, ==

function simplify(e::JuMP.GenericAffExpr{T, Variable}) where T
    coeffs = Dict{Variable, T}()
    for i in eachindex(e.vars)
        v, c = e.vars[i], e.coeffs[i]
        if c != 0
            coeffs[v] = get(coeffs, v, zero(T)) + c
        end
    end
    AffExpr(collect(keys(coeffs)), collect(values(coeffs)), e.constant)
end

Narg{N} = Tuple{Vararg{Any, N}}

immutable Conditional{Op, Args<:Tuple}
    op::Op
    args::Args
    hash::UInt
    
    function Conditional{Op, Args}(op::Op, args::Args) where {Op, Args}
        new(op, args, _hash(args, hash(op)))
    end
end

nan_to_null(x) = isnan(x) ? Nullable{typeof(x)}() : Nullable{typeof(x)}(x)

normalize(op, args) = op, args

function Conditional(op, args)
    norm_op, norm_args = normalize(op, args)
    Conditional{typeof(norm_op), typeof(norm_args)}(norm_op, norm_args)
end

normalize(op::typeof(<=), args::Narg{2}) = (<=, (simplify(args[1] - args[2]), 0))
normalize(op::typeof(>=), args::Narg{2}) = (<=, (simplify(args[2] - args[1]), 0))
normalize(op::typeof(==), args::Narg{2}) = (==, (simplify(args[1] - args[2]), 0))

"""
Like JuMP.getvalue, but returns a Nullable{T}() for unset variables instead
of throwing a warning
"""
function _getvalue(x::JuMP.GenericAffExpr{T, Variable}) where {T}
    result::Nullable{T} = x.constant
    for i in eachindex(x.coeffs)
        result = result .+ x.coeffs[i] .* _getvalue(x.vars[i])
    end
    result
end

_getvalue(x::Variable) = nan_to_null(JuMP._getValue(x))
_getvalue(x::Number) = nan_to_null(x)
_getvalue(c::Conditional{typeof(<=), <:Narg{2}}) = _getvalue(c.args[1]) .- _getvalue(c.args[2])
_getvalue(c::Conditional{typeof(>=), <:Narg{2}}) = _getvalue(c.args[2]) .- _getvalue(c.args[1])
_getvalue(c::Conditional{typeof(==), <:Narg{2}}) = abs.(_getvalue(c.args[1]) .- _getvalue(c.args[2]))
_getvalue(c::Conditional{typeof(&)}) = maximum(x -> _getvalue.(x), c.args)

(==)(c1::Conditional, c2::Conditional) = (c1.op == c2.op) && (c1.args == c2.args)

# work-around because JuMP doesn't properly define hash()
function _hash(x::JuMP.GenericAffExpr, h::UInt)
    h = hash(x.constant, h)
    for v in x.vars
        h = hash(v, h)
    end
    for c in x.coeffs
        h = hash(c, h)
    end
    h
end

_hash(x, h::UInt) = hash(x, h)

@unroll function _hash(args::Tuple, h::UInt)
    @unroll for arg in args
        h = _hash(arg, h)
    end
    return h
end

hash(c::Conditional, h::UInt) = hash(c.hash, h)
    
# function hash(c::Conditional, h::UInt)
#     h = hash(c.op, h)
#     return _hash(c.args, h)
# end

@interface ConditionalIfc(self::Conditional) begin
    _getvalue()::Nullable{Float64} = _getvalue(self)
    hash(h::UInt)::UInt = hash(self, hash(ConditionalIfc, h))
    (==)(other::ConditionalIfc)::Bool = (hash(self, hash(ConditionalIfc)) == hash(other)) && (self == other.self)
end

Base.show(io::IO, ci::ConditionalIfc) = print(io, "Interface: ($(ci.self))")

end


cj2

In [99]:
m = Model(solver=GurobiSolver())
@variable m -1 <= x <= 1
c1 = cj2.Conditional(<=, (x, 0))
c2 = cj2.Conditional(==, (x, 1))
c3 = cj2.Conditional(<=, (x + 5, 5))
ci1 = cj2.ConditionalIfc(c1)
ci2 = cj2.ConditionalIfc(c2)
ci3 = cj2.ConditionalIfc(c3)

@test c1 != c2
@test c2 != c3
@test c1 == c3
@test c1 !== c3
@test hash(c1) == hash(c3)
@test hash(c1) != hash(c2)

@test ci1 != ci2
@test ci2 != ci3
@test ci1 == ci3
@test hash(ci1) == hash(ci3)
@test hash(ci1) != hash(ci2)
@test hash(ci1) != hash(c1)

Test Passed


In [75]:
c1

cj2.Conditional{Base.#<=,Tuple{JuMP.GenericAffExpr{Float64,JuMP.Variable},Int64}}(<=, (x, 0), 0xf8ec7f62ade81d3c)

In [76]:
c3

cj2.Conditional{Base.#<=,Tuple{JuMP.GenericAffExpr{Float64,JuMP.Variable},Int64}}(<=, (x, 0), 0x247488fcd21b43a7)

In [72]:
c1 == c3

true

In [100]:
@benchmark $ci1 == $ci2

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     65.322 ns (0.00% GC)
  median time:      66.248 ns (0.00% GC)
  mean time:        70.145 ns (0.00% GC)
  maximum time:     326.171 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     979

In [101]:
@benchmark $ci1 == $ci1

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  1
  --------------
  minimum time:     646.252 ns (0.00% GC)
  median time:      667.055 ns (0.00% GC)
  mean time:        717.001 ns (0.00% GC)
  maximum time:     3.911 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     163

In [65]:
@benchmark $ci == $ci

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     52.267 ns (0.00% GC)
  median time:      53.059 ns (0.00% GC)
  mean time:        56.460 ns (0.00% GC)
  maximum time:     235.061 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     986

In [52]:
@benchmark hash($c)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     5.266 ns (0.00% GC)
  median time:      5.786 ns (0.00% GC)
  mean time:        7.108 ns (0.00% GC)
  maximum time:     75.407 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     999

In [53]:
@benchmark hash($ci)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     30.061 ns (0.00% GC)
  median time:      30.320 ns (0.00% GC)
  mean time:        31.701 ns (0.00% GC)
  maximum time:     161.445 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     995

In [39]:
v = [ci, ci, ci]
y = hash.(v)

3-element Array{UInt64,1}:
 0x950c4fdc2d2dc2f3
 0x950c4fdc2d2dc2f3
 0x950c4fdc2d2dc2f3

In [40]:
@benchmark $y .= hash.($v)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     295.435 ns (0.00% GC)
  median time:      306.559 ns (0.00% GC)
  mean time:        350.485 ns (0.00% GC)
  maximum time:     1.622 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     262